# Router 주문 응대 Classify Agent 튜토리얼


Router 주문 응대 Classify Agent는 고객의 주문 요청을 분석하여 메뉴 카테고리별로 분류하고 적절한 처리 로직으로 라우팅하는 시스템이다. 카페, 레스토랑 등에서 다양한 메뉴 카테고리를 효율적으로 처리할 수 있다.

기본 작동 원리는 다음과 같다:
1. 고객의 주문 메시지를 입력받는다
2. LLM이 주문 내용을 분석하여 메뉴 카테고리를 분류한다
3. 분류된 카테고리에 따라 해당 주문 처리 함수를 실행한다
4. 주문 확인 메시지와 함께 처리 결과를 반환한다

이 시스템은 음료, 디저트, 식사 등 다양한 메뉴 카테고리를 자동으로 구분하여 처리할 수 있다.

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

True

In [3]:
from openai import OpenAI
import json
from datetime import datetime

# OpenAI 클라이언트 초기화
client = OpenAI()

In [4]:
# 메뉴 카테고리별 메뉴 목록
menu_database = {
    "음료": {
        "아메리카노": 4500,
        "카페라떼": 5000,
        "카푸치노": 5000,
        "아이스티": 4000,
        "주스": 5500
    },
    "디저트": {
        "케이크": 6000,
        "마카롱": 3000,
        "쿠키": 2500,
        "와플": 7000,
        "타르트": 6500
    },
    "식사": {
        "샌드위치": 8000,
        "샐러드": 9000,
        "파스타": 12000,
        "리조또": 13000,
        "피자": 15000
    }
}

In [5]:
def classify_order(order_text):
    """주문 내용을 분석하여 메뉴 카테고리를 분류한다"""
    
    prompt = f"""
    다음 주문 내용을 분석하여 가장 적절한 메뉴 카테고리를 선택하라.
    
    주문 내용: {order_text}
    
    카테고리: 음료, 디저트, 식사
    
    카테고리 하나만 답변하라.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    
    category = response.choices[0].message.content.strip()
    return category

In [6]:
def extract_order_items(order_text, category):
    """주문 내용에서 구체적인 메뉴와 수량을 추출한다"""
    
    menu_list = list(menu_database[category].keys())
    
    prompt = f"""
    다음 주문 내용에서 메뉴와 수량을 추출하라.
    
    주문 내용: {order_text}
    
    사용 가능한 메뉴: {', '.join(menu_list)}
    
    반드시 아래 JSON 형식으로만 답변하라. 다른 설명이나 텍스트는 포함하지 마라:
    {{
        "items": [
            {{"menu": "메뉴명", "quantity": 수량}}
        ]
    }}
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    
    # 응답에서 JSON 추출 및 정제
    content = response.choices[0].message.content.strip()
    
    # 마크다운 코드 블록 제거
    content = content.replace('```json', '').replace('```', '').strip()
    
    # JSON 파싱
    result = json.loads(content)
    return result["items"]

In [7]:
def process_beverage_order(items):
    """음료 주문을 처리한다"""
    
    print("[음료 주문 처리 시스템]")
    
    total_price = 0
    order_summary = []
    
    for item in items:
        menu = item["menu"]
        quantity = item["quantity"]
        price = menu_database["음료"].get(menu, 0)
        subtotal = price * quantity
        total_price += subtotal
        
        order_summary.append(f"  - {menu} x {quantity}개 = {subtotal:,}원")
    
    return order_summary, total_price

In [8]:
def process_dessert_order(items):
    """디저트 주문을 처리한다"""
    
    print("[디저트 주문 처리 시스템]")
    
    total_price = 0
    order_summary = []
    
    for item in items:
        menu = item["menu"]
        quantity = item["quantity"]
        price = menu_database["디저트"].get(menu, 0)
        subtotal = price * quantity
        total_price += subtotal
        
        order_summary.append(f"  - {menu} x {quantity}개 = {subtotal:,}원")
    
    return order_summary, total_price

In [9]:
def process_meal_order(items):
    """식사 주문을 처리한다"""
    
    print("[식사 주문 처리 시스템]")
    
    total_price = 0
    order_summary = []
    
    for item in items:
        menu = item["menu"]
        quantity = item["quantity"]
        price = menu_database["식사"].get(menu, 0)
        subtotal = price * quantity
        total_price += subtotal
        
        order_summary.append(f"  - {menu} x {quantity}개 = {subtotal:,}원")
    
    return order_summary, total_price

In [10]:
def route_order_processing(category, items):
    """카테고리에 따라 적절한 주문 처리 함수로 라우팅한다"""
    
    # 카테고리별 처리 함수 매핑
    router = {
        "음료": process_beverage_order,
        "디저트": process_dessert_order,
        "식사": process_meal_order
    }
    
    # 해당 카테고리의 처리 함수 실행
    process_func = router.get(category)
    order_summary, total_price = process_func(items)
    
    return order_summary, total_price

In [11]:
def order_classify_agent(order_text):
    """Router 주문 응대 Classify Agent의 전체 파이프라인을 실행한다"""
    
    print(f"고객 주문: {order_text}\n")
    
    # 1단계: 카테고리 분류
    category = classify_order(order_text)
    print(f"분류된 카테고리: {category}\n")
    
    # 2단계: 주문 항목 추출
    items = extract_order_items(order_text, category)
    print(f"추출된 주문 항목: {items}\n")
    
    # 3단계: 카테고리별 주문 처리
    order_summary, total_price = route_order_processing(category, items)
    
    # 4단계: 주문 확인 메시지 생성
    print("\n===== 주문 확인서 =====")
    print(f"주문 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"카테고리: {category}")
    print("\n주문 내역:")
    for summary in order_summary:
        print(summary)
    print(f"\n총 결제 금액: {total_price:,}원")
    print("=====================\n")
    
    # 결과 반환
    result = {
        "order_text": order_text,
        "category": category,
        "items": items,
        "total_price": total_price
    }
    
    return result

In [12]:
# 테스트 실행
test_orders = [
    "아메리카노 2잔이랑 카페라떼 1잔 주세요",
    "케이크 1개랑 마카롱 3개 포장해주세요",
    "파스타 2인분이랑 샐러드 1개 주문할게요"
]

results = []

for order in test_orders:
    print("=" * 80)
    result = order_classify_agent(order)
    results.append(result)
    print("=" * 80)
    print()

고객 주문: 아메리카노 2잔이랑 카페라떼 1잔 주세요

분류된 카테고리: 음료

추출된 주문 항목: [{'menu': '아메리카노', 'quantity': 2}, {'menu': '카페라떼', 'quantity': 1}]

[음료 주문 처리 시스템]

===== 주문 확인서 =====
주문 시간: 2025-10-24 22:06:13
카테고리: 음료

주문 내역:
  - 아메리카노 x 2개 = 9,000원
  - 카페라떼 x 1개 = 5,000원

총 결제 금액: 14,000원


고객 주문: 케이크 1개랑 마카롱 3개 포장해주세요

분류된 카테고리: 디저트

추출된 주문 항목: [{'menu': '케이크', 'quantity': 1}, {'menu': '마카롱', 'quantity': 3}]

[디저트 주문 처리 시스템]

===== 주문 확인서 =====
주문 시간: 2025-10-24 22:06:15
카테고리: 디저트

주문 내역:
  - 케이크 x 1개 = 6,000원
  - 마카롱 x 3개 = 9,000원

총 결제 금액: 15,000원


고객 주문: 파스타 2인분이랑 샐러드 1개 주문할게요

분류된 카테고리: 식사

추출된 주문 항목: [{'menu': '파스타', 'quantity': 2}, {'menu': '샐러드', 'quantity': 1}]

[식사 주문 처리 시스템]

===== 주문 확인서 =====
주문 시간: 2025-10-24 22:06:18
카테고리: 식사

주문 내역:
  - 파스타 x 2개 = 24,000원
  - 샐러드 x 1개 = 9,000원

총 결제 금액: 33,000원




In [13]:
# 전체 매출 요약
print("\n===== 전체 주문 요약 =====")
total_revenue = sum(r["total_price"] for r in results)
category_count = {}

for result in results:
    category = result["category"]
    category_count[category] = category_count.get(category, 0) + 1

print(f"총 주문 건수: {len(results)}건")
print(f"총 매출: {total_revenue:,}원")
print("\n카테고리별 주문 건수:")
for category, count in category_count.items():
    print(f"  - {category}: {count}건")


===== 전체 주문 요약 =====
총 주문 건수: 3건
총 매출: 62,000원

카테고리별 주문 건수:
  - 음료: 1건
  - 디저트: 1건
  - 식사: 1건
